In [26]:
import pandas as pd
import numpy as np
import os

In [27]:
path = r'/Users/pgtips/Desktop/Instacart Basket Analysis/02.Data/Original Data'

# Orders_df

In [28]:
df_orders = pd.read_csv(os.path.join(path, 'orders.csv'), index_col = False)

In [29]:
df_orders.shape

(3421083, 7)

In [30]:
df_orders.head(1)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN


In [31]:
df_ords = df_orders.drop(columns = ['eval_set'])

In [32]:
df_ords.rename(columns = {'order_dow' : 'orders_day_of_week'}, inplace = True)

#### We decided to drop the column 'eval_set' as its has no value for the analysis

In [33]:
df_ords.dtypes

order_id                    int64
user_id                     int64
order_number                int64
orders_day_of_week          int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

##### We have to change the dtype of 'order_id' and 'user_id' as its a unique combination of numbers, to make sure that the system doesnt see it as numbers

In [34]:
df_ords['order_id'] = df_ords['order_id'].astype(str)
df_ords['user_id'] = df_ords['user_id'].astype(str)

In [35]:
df_ords.dtypes

order_id                   object
user_id                    object
order_number                int64
orders_day_of_week          int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

##### Since changing the dtype we can check for mixed dtypes with the code below

In [36]:
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

### Missing Values

In [37]:
df_ords.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

##### We found over 200k missing values, due to the context of the column we guess that these customers never order from us again, so that we actually have information that around 16% of customers dont return!

In [38]:
df_ords['days_since_prior_order'].value_counts()

30.0    369323
7.0     320608
6.0     240013
4.0     221696
3.0     217005
5.0     214503
2.0     193206
8.0     181717
1.0     145247
9.0     118188
14.0    100230
10.0     95186
13.0     83214
11.0     80970
12.0     76146
0.0      67755
15.0     66579
16.0     46941
21.0     45470
17.0     39245
20.0     38527
18.0     35881
19.0     34384
22.0     32012
28.0     26777
23.0     23885
27.0     22013
24.0     20712
25.0     19234
29.0     19191
26.0     19016
Name: days_since_prior_order, dtype: int64

### Duplicates

In [39]:
df_dups_ords = df_ords[df_ords.duplicated()]

In [40]:
df_dups_ords.shape

(0, 6)

##### No duplicates been found

### Descriptive Stat

In [41]:
df_ords.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.42108e+06,3.42108e+06,3.42108e+06,3.21487e+06
mean,17.1549,2.77622,13.452,11.1148
std,17.7332,2.04683,4.22609,9.20674
min,1,0,0,0
25%,5,1,10,4
50%,11,3,13,7
75%,23,5,16,15
max,100,6,23,30


In [42]:
df_outliner = df_ords[df_ords['order_number'] >= 70]

In [43]:
from scipy import stats

In [44]:
df_outliner.shape

(80139, 6)

In [45]:
df_ords.shape

(3421083, 6)

##### the descriptive stats for order_number seems to have some outline, if we add 3 times the std to the mean (Gausbell / Normaldistribution) we have roughly 70 orders, but the max is close to 100. About 2% of the df could be considered outliners.
Our hypothsis is supported as the 75% ouratile is just 1/4 of the max which represents the 100% quartile!

In [46]:
df_ords = df_ords[df_ords['order_number'] <= 70]

In [47]:
df_ords.shape

(3345357, 6)

In [48]:
df_ords.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.34536e+06,3.34536e+06,3.34536e+06,3.13915e+06
mean,15.6729,2.77388,13.461,11.3021
std,14.8584,2.0495,4.22524,9.22514
min,1,0,0,0
25%,5,1,10,4
50%,10,3,13,8
75%,22,5,16,16
max,70,6,23,30


In [49]:
df_ords.to_csv(r'/Users/pgtips/Desktop/Instacart Basket Analysis/05 Sent to Clint/Prepared Data/orders.csv', index = False)